<a href="https://colab.research.google.com/github/t-willi/Simula/blob/main/get_predictions_for_s/sn/snpTOP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install wandb
!pip install ecg_plot


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.9 MB 33.3 MB/s 
     |████████████████████████████████| 182 kB 55.8 MB/s 
     |████████████████████████████████| 166 kB 61.1 MB/s 
     |████████████████████████████████| 63 kB 2.0 MB/s 
     |████████████████████████████████| 166 kB 62.4 MB/s 
     |████████████████████████████████| 162 kB 54.5 MB/s 
     |████████████████████████████████| 162 kB 57.4 MB/s 
     |████████████████████████████████| 158 kB 57.2 MB/s 
     |████████████████████████████████| 157 kB 49.6 MB/s 
     |████████████████████████████████| 157 kB 56.8 MB/s 
     |████████████████████████████████| 157 kB 59.5 MB/s 
     |████████████████████████████████| 157 kB 53.6 MB/s 
     |████████████████████████████████| 157 kB 56.1 MB/s 
     |████████████████████████████████| 157 kB 67.4 MB/s 
     |████████████████████████████████| 157 kB 59.9 MB/s 
     |███████████████████████████

In [2]:
import pathlib
from pathlib import Path
# current_position=pathlib.PurePath(__file__)
# dir=current_position.parent
# main_folder=dir.joinpath("main_folder")
current_position=pathlib.PurePath("/content/")
main_folder=current_position.joinpath("main_folder")
Path(main_folder).mkdir(parents=False,exist_ok = True)
#create folder and directory for artifacts
artifact_dir=main_folder.joinpath("artifacts")
Path(artifact_dir).mkdir(parents=False,exist_ok = True)
#create folder and directory for train_data
train_dir=main_folder.joinpath("train_dir")
Path(train_dir).mkdir(parents=False,exist_ok = True)
#create folder and directory for ecg_files
ecg_dir=main_folder.joinpath("ecg")
Path(ecg_dir).mkdir(parents=False,exist_ok = True)
#create folder and directory for saved model sate dicts
model_dir=main_folder.joinpath("model")
Path(model_dir).mkdir(parents=False,exist_ok = True)


In [5]:
import wandb
import pandas as pd
import torch
from torch import nn
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from torch.utils.data import Dataset, DataLoader
import glob
import torch.optim as optim
from random import shuffle
from tqdm.auto import tqdm
import requests
import zipfile
from pathlib import Path

if torch.cuda.is_available()==True:
  device="cuda:0"
else:
  device ="cpu"

apikey = "7a8ee9d41cc2d51eb77fd795e14f74a215e63c2d"
api = wandb.Api()
test_artifact = api.artifact('ecg_simula/upload_Model_test_data/Test_data:v0', type='dataset')
test_artifact.download("/content/artifacts/test_data")
pTOP_1_7_syn = api.artifact('ecg_simula/AE_pTOP_serverrun_synthetic_data/Model:v6', type='Model')
## pTOP_1_7_norm comes from training of pTOP on PTB_normal data with pretraining from ecg_simula/AE_pTOP_serverrun_synthetic_data/Model:v3
pTOP_1_7_norm = api.artifact('ecg_simula/pTOP_PTB_normal/Model:v54', type='Model')
## model trained with pathological data
pTOP_1_7_patho = api.artifact('ecg_simula/pTOP_PTB_patho/Model:v38', type='Model')
## download model states 
#models for ecg prediciton
pTOP_1_7_syn.download("/content/artifacts/pTOP_1_7_syn")
pTOP_1_7_norm.download("/content/artifacts/pTOP_1_7_norm")
pTOP_1_7_patho.download("/content/artifacts/pTOP_1_7_patho")

torch.manual_seed(42)

wandb:   1 of 1 files downloaded.  
wandb:   1 of 1 files downloaded.  
wandb:   1 of 1 files downloaded.  
wandb:   1 of 1 files downloaded.  


In [9]:
# previous_model = api.artifact('ecg_simula/AE_pTOP_LS_optim/Model:v8', type='Model')
# previous_model.download()

def request(path=None,name=None):
  import requests
  from pathlib import Path
  request = requests.get(path)
  name=name+".py"
  with open(name,"wb") as f:
    f.write(request.content)

unzip_git_dir="https://raw.githubusercontent.com/t-willi/NeuralNetworks/main/unzip.py"
get_pred="https://raw.githubusercontent.com/t-willi/NeuralNetworks/main/get_pred_12Lead.py"
plt_ECG_git_dir="https://raw.githubusercontent.com/t-willi/NeuralNetworks/main/plot_ECG_12Lead.py"
Dataloader_syn_1to7_git_dir="https://raw.githubusercontent.com/t-willi/NeuralNetworks/main/dataloader_syn_1to7.py"
Dataloader_normal_1to7_git_dir="https://raw.githubusercontent.com/t-willi/NeuralNetworks/main/dataloader_PTB_normal_1to7.py"

request(unzip_git_dir,"Unzip")
from Unzip import unzip

#request(pTOP_git_dir,"pTOP")
#from pTOP import *
request(Dataloader_syn_1to7_git_dir,"dataset_and_loader")
from dataset_and_loader import make_loader as ml
request(Dataloader_syn_1to7_git_dir,"syn1_7")
from syn1_7 import Custom_dataset as CD_syn_1_7
request(Dataloader_normal_1to7_git_dir,"normal1_7")
from normal1_7 import Custom_dataset_PTB as CD_normal_1_7
#download prediction generator
request(get_pred,"get_predictions")
from get_predictions import get_pred_12lead as get_pred
#download ECG plotter
request(plt_ECG_git_dir,"plot_ECG")
from plot_ECG import plotECG_12Lead as plotECG

unzip(save_path="/content/test_data",zip_path="/content/artifacts/test_data/ecg_prediction_data.zip",reload=True)


Did not find /content/test_data directory, creating one...
Unzipping data to folder...
unzip is finished


In [65]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data

class Transpose1dLayer(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride, padding=11, upsample=None, output_padding=1):
        super(Transpose1dLayer, self).__init__()
        self.upsample = upsample
        self.upsample_layer = torch.nn.Upsample(scale_factor=upsample)
        reflection_pad = kernel_size // 2
        self.reflection_pad = nn.ConstantPad1d(reflection_pad, value=0)
        self.conv1d = torch.nn.Conv1d(in_channels, out_channels, kernel_size, stride)
        self.Conv1dTrans = nn.ConvTranspose1d(in_channels, out_channels, kernel_size, stride, padding, output_padding)

    def forward(self, x):
        if self.upsample:
            #x = torch.cat((x, in_feature), 1)
            return self.conv1d(self.reflection_pad(self.upsample_layer(x)))
        else:
            return self.Conv1dTrans(x)


class pTOP_1to7(nn.Module):
    def __init__(self,latent_dim=100, post_proc_filt_len=512,upsample=True):
        super(pTOP_1to7, self).__init__()
        # "Dense" is the same meaning as fully connection.
        stride = 4
        if upsample:
            stride = 1
            upsample = 5
        # if upsample is anything but none Transpose1dLayer will do
        # self.conv1d(self.reflection_pad(self.upsample_layer(x)))
        # which is a 1d convolution on padded and upsampled data x
        self.deconv_1 = Transpose1dLayer(250 , 250, 25, stride, upsample=upsample)
        self.deconv_2 = Transpose1dLayer(250, 150, 25, stride, upsample=upsample)
        self.deconv_3 = Transpose1dLayer(150, 50, 25, stride, upsample=upsample)
        self.deconv_4 = Transpose1dLayer( 50, 25, 25, stride, upsample=2)
        self.deconv_5 = Transpose1dLayer( 25, 10, 25, stride, upsample=upsample)
        self.deconv_6 = Transpose1dLayer(  10, 7, 25, stride, upsample=2)


        #new convolutional layers
        self.conv_1 = nn.Conv1d(1, 10, 25, stride=2, padding=25 // 2)
        self.conv_2 = nn.Conv1d(10, 25, 25, stride=5, padding= 25 // 2)
        self.conv_3 = nn.Conv1d(25, 50 , 25, stride=2, padding= 25 // 2)
        self.conv_4 = nn.Conv1d(50, 150 , 25, stride=5, padding= 25 // 2)
        self.conv_5 = nn.Conv1d(150, 250 , 25, stride=5, padding= 25 // 2)
        self.conv_6 = nn.Conv1d(250, 250 , 25, stride=5, padding= 25 // 2)
        self.flatt = nn.Flatten()
        self.linear1 = nn.Linear(500,100)
        self.linear2 = nn.Linear(100,500)

        for m in self.modules():
            if isinstance(m, nn.ConvTranspose1d) or isinstance(m, nn.Linear):
                nn.init.kaiming_normal_(m.weight.data)

    def forward(self, x, LS=False):
        self.LS=LS
        if x.ndim==2:
          x=x.unsqueeze(0)
        x = F.leaky_relu(self.conv_1(x)) #(1,1,5000 --> 1, 10, 2500)
        x = F.leaky_relu(self.conv_2(x)) #( --> 1, 25, 500)
        x = F.leaky_relu(self.conv_3(x)) #(--> 1, 50, 250)
        x = F.leaky_relu(self.conv_4(x)) # --> 1, 150, 50)
        x = F.leaky_relu(self.conv_5(x)) #(--> 1, 250, 10)
        x = F.leaky_relu(self.conv_6(x)) #(--> 1, 250, 2)-->flatten into (1,500)), then to linear ((1,100)), and then back
        x = self.flatt(x) # (1,500)
        LS = self.linear1(x) #(1,100)
        if self.LS is True:
          return LS
        x = self.linear2(LS) #(1,500)
        zero_dim=x.shape[0]
        x=torch.reshape(x,(zero_dim,250,2)) #1(1,250,2)
        x = F.relu(self.deconv_1(x)) #(--> 1, 250, 10)
        x = F.relu(self.deconv_2(x)) #(--> 1, 150, 50)
        x = F.relu(self.deconv_3(x)) #( --> 1, 50, 250)
        x = F.relu(self.deconv_4(x)) #(--> 1, 25, 500)
        x = F.relu(self.deconv_5(x)) #(--> 1, 10, 2500)
        x = torch.tanh(self.deconv_6(x)) #(1, 7, 5000)
        x=x.squeeze()
        return x

model_syn=pTOP_1to7().to(device)
model_norm=pTOP_1to7().to(device)
model_patho=pTOP_1to7().to(device)
# PATH="/content/artifacts/Model:v3/model"
# model.load_state_dict(torch.load(PATH))

Create data to run trough the pTOP7to7 for feature extraction. A sample from syn data will be run trough trained pTOP1to7 models, the 7 lead output will be run trough a pTOP7to7 model and its latent space compared to the latent space of the real 7 leads.


In [26]:
import glob
import pandas as pd
import torch
class Custom_dataset_syn():
    def __init__(self, data_dir,max_value=5011,column=3,split=True,target="train",size=1):
      #get all files from directory loaded in all_files list
      self.column=column
      self.max_value=max_value
      self.size=size
      #should shuffle the data here?
      self.files = glob.glob(data_dir + '/*.csv')
      self.len=int((len(self.files))*self.size)
      #print(f"len:{self.len}")
      self.cut1=int(self.len*0.8)
      #print(f"cut1:{self.cut1}")
      self.cut2=int(self.len*0.9)
      #print(f"cut2:{self.cut2}")
      self.train_files=self.files[0:self.cut1]
      self.test_files=self.files[self.cut1:self.cut2]
      self.val_files=self.files[self.cut2:self.len]
      self.target=target
      self.split=split

    def __len__(self):
      if self.split is True:
        if self.target is "train":
          return len(self.train_files)
        if self.target is "test":
          return len(self.test_files)
        if self.target is "val":
          return len(self.val_files)
      if self.split is not True:
        return len(self.files)

    def __getitem__(self,idx):
      header=["I", "II", "v1", "v2", "v3", "v4", "v5", "v6"]
      #turn list of dataframes into Tensor
      if self.split is True:
        if self.target is "train":
          temp_df=pd.read_csv(self.train_files[idx],index_col=0,header=0,names=header)
        if self.target is "test":
          temp_df=pd.read_csv(self.test_files[idx],index_col=0,header=0,names=header)
        if self.target is "val":
          temp_df=pd.read_csv(self.val_files[idx],index_col=0,header=0,names=header)
      if self.split is not True:
        temp_df=pd.read_csv(self.files[idx],index_col=0,header=0,names=header)
      temp_df/=self.max_value
      #load input tensor
      
      temp_list_in=temp_df.loc[:,"I"]
      #temp_list_in=normalize([temp_list_in], norm="max")
      temp_tensor_in = torch.tensor(temp_list_in,dtype=torch.float32)
      temp_tensor_in=temp_tensor_in.unsqueeze(0)
      #load label Tensor
      temp_list_out=temp_df.loc[:,["II","v1","v2","v3","v4","v5","v6"]].values
      #temp_list_out=normalize([temp_list_out], norm="max")
      temp_tensor_out=torch.tensor(temp_list_out,dtype=torch.float32)
      temp_tensor_out=temp_tensor_out.T
      #combine input and label and output
      temp_tensor_pair= temp_tensor_in,temp_tensor_out
      return temp_tensor_pair

def make_loader(dataset,batch_size):
  from torch.utils.data import DataLoader
  loader = DataLoader(dataset,
                      batch_size=batch_size,
                      shuffle=True,
                      drop_last=True
                      )
  return loader

In [34]:
import glob
import pandas as pd
import torch
class Custom_dataset_PTB():
    def __init__(self, data_dir,max_value=33,column=3,split=True,target="train",size=1):
      #get all files from directory loaded in all_files list
      self.column=column
      self.max_value=max_value
      self.size=size
      #should shuffle the data here?
      self.files = glob.glob(data_dir + '/*.csv')
      self.len=int((len(self.files))*self.size)
      #print(f"len:{self.len}")
      self.cut1=int(self.len*0.8)
      #print(f"cut1:{self.cut1}")
      self.cut2=int(self.len*0.9)
      #print(f"cut2:{self.cut2}")
      self.train_files=self.files[0:self.cut1]
      self.test_files=self.files[self.cut1:self.cut2]
      self.val_files=self.files[self.cut2:self.len]
      self.target=target
      self.split=split

    def __len__(self):
      if self.split is True:
        if self.target is "train":
          return len(self.train_files)
        if self.target is "test":
          return len(self.test_files)
        if self.target is "val":
          return len(self.val_files)
      if self.split is not True:
        return len(self.files)

    def __getitem__(self,idx):
      header=["I", "II", "III", "aVF", "aVR", "aVL", "v1", "v2", "v3", "v4", "v5", "v6"]
      #turn list of dataframes into Tensor
      if self.split is True:
        if self.target is "train":
          temp_df=pd.read_csv(self.train_files[idx],index_col=0,header=0,names=header)
        if self.target is "test":
          temp_df=pd.read_csv(self.test_files[idx],index_col=0,header=0,names=header)
        if self.target is "val":
          temp_df=pd.read_csv(self.val_files[idx],index_col=0,header=0,names=header)
      if self.split is not True:
        temp_df=pd.read_csv(self.files[idx],index_col=0,header=0,names=header)
      temp_df/=self.max_value
      #load input tensor
      
      temp_list_in=temp_df.loc[:,"I"]
      #temp_list_in=normalize([temp_list_in], norm="max")
      temp_tensor_in = torch.tensor(temp_list_in,dtype=torch.float32)
      temp_tensor_in=temp_tensor_in.unsqueeze(0)
      #load label Tensor
      temp_list_out=temp_df.loc[:,["II","v1","v2","v3","v4","v5","v6"]].values
      #temp_list_out=normalize([temp_list_out], norm="max")
      temp_tensor_out=torch.tensor(temp_list_out,dtype=torch.float32)
      temp_tensor_out=temp_tensor_out.T
      #combine input and label and output
      temp_tensor_pair= temp_tensor_in,temp_tensor_out
      return temp_tensor_pair

def make_loader(dataset,batch_size):
  from torch.utils.data import DataLoader
  loader = DataLoader(dataset,
                      batch_size=batch_size,
                      shuffle=True,
                      drop_last=True
                      )
  return loader

In [48]:
syn_test_data=Custom_dataset_syn("/content/test_data/Syn",split=False)
STTC_test_data=Custom_dataset_PTB("/content/test_data/STTC",split=False)
Norm_test_data=Custom_dataset_PTB("/content/test_data/Norm",split=False)
MI_test_data=Custom_dataset_PTB("/content/test_data/MI",split=False)
HYP_test_data=Custom_dataset_PTB("/content/test_data/HYP",split=False)
CD_test_data=Custom_dataset_PTB("/content/test_data/CD",split=False)
datasets={"syn_test_data":syn_test_data,
          "STTC_test_data":STTC_test_data,
          "Norm_test_data":Norm_test_data,
          "MI_test_data":MI_test_data,
          "HYP_test_data":HYP_test_data,
          "CD_test_data":CD_test_data
          }


In [70]:

#set up 1to7 models for prediction
syn1_7_path="/content/artifacts/pTOP_1_7_syn/model"
model_syn.load_state_dict(torch.load(syn1_7_path, map_location=device))
norm1_7_path="/content/artifacts/pTOP_1_7_norm/model"
model_norm.load_state_dict(torch.load(norm1_7_path, map_location=device))
patho1_7_path="/content/artifacts/pTOP_1_7_patho/model"
model_patho.load_state_dict(torch.load(patho1_7_path, map_location=device))
models={"model_syn":model_syn,
        "model_norm":model_norm,
        "model_patho":model_patho
        }

In [ ]:
#get prediction from dataset for synthetic data, this is for testing purposes to see that the ecg looks right.
dataset="HYP_test_data"
for model in models:
  for k,files in enumerate(datasets[dataset]):
    temp_in,temp_out=get_pred_12lead(random=False,Set=files,model=models[model],upscale=33)
    combined_df=plotECG(temp_in,temp_out,title=f"{k}_{model}",path=f"/content/ecg_predictions/{dataset}",scale=None)



In [78]:
import shutil
shutil.make_archive("ecg_predictions", 'zip', "/content/ecg_predictions")


'/content/ecg_predictions.zip'